In [ ]:
# Importing libraries to collect and import data:
import json
import requests
import pickle

# Importing libraries to work with/explore data:
import pandas as pd
import numpy as np
import datetime, time
from sklearn import preprocessing

# Importing libraries to create visualizations:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
# import itertools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define function to gather keys:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'

In [ ]:
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [ ]:
city_list = {
    'New York City': borough_nyc,
    'Los Angeles': los_angeles,
    'Chicago': chicago_city,
    'Houston': houston_city
             }

In [ ]:
city_lists = {city: [[] for _ in range(len(area))] for city, area in city_list.items()}

In [ ]:
nyc_list = pd.read_csv('14805_nyc.csv')

In [ ]:
la_list =  pd.read_csv('9298_la.csv')

In [ ]:
chicago_list = pd.read_csv('5259_chicago.csv')

In [ ]:
houston_list = pd.read_csv('5332_houston.csv')

In [ ]:
len(nyc_list['neighborhood'].unique())

82

# API framework

In [ ]:
city_dataframes = {
    'New York City': nyc_list,
    'Los Angeles': la_list,
    'Chicago': chicago_list,
    'Houston': houston_list
             }

In [ ]:
def api_data(city, area):
    city_lists = {city: [[] for _ in area]}  # Initialize the city_lists dictionary

    for x in range(len(area)):
        print('---------------------------------------------')
        print(f'Gathering Data for {area[x]}')
        print('---------------------------------------------')

        for y in range(12):
            location = area[x] + f', {city}'
            term = "Restaurants"
            search_limit = 20
            offset = 20 * y
            categories = "(restaurants, All)"
            sort_by = 'distance'
            url_params = {
                'location': location.replace(' ', '+'),
                'term': term,
                'limit': search_limit,
                'offset': offset,
                'categories': categories,
                'sort_by': sort_by
            }

            response = requests.get(url, headers=headers, params=url_params)

            # Check if the response is None or if the status code is not 200
            if response is None or response.status_code != 200:
                print(f'Error or no results for {area[x]}, skipping to next area...')
                break  # Exit the loop over y in range(12)

            print(f'***** {area[x]} Restaurants #{offset+1} - #{offset+search_limit} .... {response}')
            city_lists[city][x].append(response.json())

        # Debugging prints
        print(response)
        print(type(response.text))
        print(response.json().keys())
        print(response.text[:1000])

    return city_lists


In [ ]:
neighborhood_la = api_data('la', los_angeles)

In [ ]:
def api_data(city, area):
    city_lists = {city: [[] for _ in area]}  # Initialize the city_lists dictionary

    for x in range(len(area)):
        print('---------------------------------------------')
        print(f'Gathering Data for {area[x]}')
        print('---------------------------------------------')

        for y in range(12):
            location = area[x] + f', {city}'
            term = "Restaurants"
            search_limit = 20
            offset = 20 * y
            categories = "(restaurants, All)"
            sort_by = 'distance'
            url_params = {
                'location': location.replace(' ', '+'),
                'term': term,
                'limit': search_limit,
                'offset': offset,
                'categories': categories,
                'sort_by': sort_by
            }

            response = requests.get(url, headers=headers, params=url_params)

            # Check if the response contains no results
            if response.status_code != 200 or len(response.json().get('businesses', [])) == 0:
                print(f'No results for {area[x]}, skipping to next area...')
                break  # Skip to the next area if no results

            print(f'***** {area[x]} Restaurants #{offset+1} - #{offset+search_limit} .... {response}')
            city_lists[city][x].append(response.json())

        # Debugging prints
        print(response)
        print(type(response.text))
        print(response.json().keys())
        print(response.text[:1000])

    return city_lists

In [ ]:
neighborhood_la = api_data('la', los_angeles)

---------------------------------------------
Gathering Data for Beverly Hills
---------------------------------------------
***** Beverly Hills Restaurants #1 - #20 ....<Response [200]>
***** Beverly Hills Restaurants #21 - #40 ....<Response [200]>
***** Beverly Hills Restaurants #41 - #60 ....<Response [200]>
***** Beverly Hills Restaurants #61 - #80 ....<Response [200]>
***** Beverly Hills Restaurants #81 - #100 ....<Response [200]>
***** Beverly Hills Restaurants #101 - #120 ....<Response [200]>
***** Beverly Hills Restaurants #121 - #140 ....<Response [200]>
***** Beverly Hills Restaurants #141 - #160 ....<Response [200]>
***** Beverly Hills Restaurants #161 - #180 ....<Response [200]>
***** Beverly Hills Restaurants #181 - #200 ....<Response [200]>
***** Beverly Hills Restaurants #201 - #220 ....<Response [200]>
***** Beverly Hills Restaurants #221 - #240 ....<Response [200]>
---------------------------------------------
Gathering Data for Bel Air
--------------------------------

In [ ]:
def dataframe_creator(city, area):
  df = pd.DataFrame()
  for x in range(len(area)):
    for y in range(12):
      # Extract JSON data from the response
      data = neighborhood_la[city][x][y].json()
      df_temp = pd.DataFrame.from_dict(data['businesses']) # Now access 'businesses' from the JSON
      df_temp.loc[:,'neighborhood'] = area[x]
      neighborhood = area[x].replace(' ', '_')
      df = pd.concat([df, df_temp], ignore_index=True) # Use pd.concat to append dataframes
    df.to_csv(f'{city}_{neighborhood}.csv')

In [ ]:
dataframe_creator('la',los_angeles)

In [ ]:
def dataframe_creator(city, area):
  df = pd.DataFrame()
  for x in range(len(area)):
    for y in range(12):
      # Extract JSON data from the response
      data = neighborhood_la[city][x][y].json()

      # Check if 'businesses' key exists and has data
      if 'businesses' in data and data['businesses']:
          df_temp = pd.DataFrame.from_dict(data['businesses'])
          df_temp.loc[:,'neighborhood'] = area[x]
          neighborhood = area[x].replace(' ', '_')
          df = pd.concat([df, df_temp], ignore_index=True)
      else:
          print(f"No businesses found for {area[x]} in request {y+1}") # Inform the user
    df.to_csv(f'{city}_{neighborhood}.csv')

# Combine restaurant information

In [ ]:
import pandas as pd
import glob

# List all CSV files
csv_files = glob.glob('*.csv')

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# Drop duplicate rows
unique_df = combined_df.drop_duplicates(['url'])

# Save the resulting DataFrame to a new CSV file
unique_df.to_csv(f'{len(unique_df)}_nyc.csv', index=False)

# NYC

In [ ]:
len(set(Manhattan)-set(nyc_list['neighborhood'].unique()))

0

In [ ]:
borough_nyc = ['Manhattan','Bronx','Brooklyn','Queens','Staten Island']

In [ ]:
# List of Manhattan neighborhoods from https://www.nyctourist.com/million-manhattan.htm
Manhattan = ['Midtown West','Greenwich Village', 'East Harlem', 'Upper East Side', 'Midtown East',
                 'Gramercy', 'Little Italy',
             'Chinatown', 'SoHo', 'Harlem',
                 'Upper West Side', 'Tribeca', 'Garment District', 'Stuyvesant Town', 'Financial District',
                 'Chelsea', 'Morningside Heights', 'Times Square', 'Murray Hill', 'East Village',
                 'Lower East Side', 'Hell\s Kitchen', 'Central Park']

In [ ]:
Bronx = ['Melrose', 'Mott Haven', 'Port Morris', 'Longwood', 'Hunts Point', 'Morrisania',
    'Crotona Park East', 'Highbridge', 'Concourse', 'Morris Heights', 'Mount Hope', 'Tremont', 'Belmont',
    'West Farms', 'Fordham', 'Bedford Park', 'Norwood', 'Riverdale', 'Kingsbridge', 'Marble Hill',
    'Soundview', 'Parkchester', 'Co-op City', 'Throgs Neck', 'Pelham Parkway', 'Morris Park',
    'Wakefield', 'Williamsbridge', 'Eastchester', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park']

In [ ]:
Queens = [
    'Astoria', 'Astoria Heights', 'Ditmars', 'Steinway', 'Little Egypt',
    'Long Island City', 'Blissville', 'Hunters Point', 'Dutch Kills',
    'Queensbridge',
    'Queensview', 'Queens West', 'Ravenswood', 'Sunnyside', 'Sunnyside Gardens',
    'Woodside', 'Little Manila', 'Boulevard Gardens', 'The Hole', 'Howard Beach',
    'Hamilton Beach', 'Howard Park', 'Lindenwood', 'Old Howard Beach', 'Ramblersville',
    'Rockwood Park', 'Ozone Park', 'South Ozone Park', 'Tudor Village', 'Richmond Hill',
    'Woodhaven', 'Briarwood', 'Corona', 'LeFrak City', 'North Corona', 'East Elmhurst',
    'LaGuardia Airport', 'Landing Lights', 'Astoria Heights', 'North Beach', 'Elmhurst',
    'Forest Hills', 'Forest Hills Gardens', 'Fresh Pond', 'Glendale', 'Jackson Heights',
    'Kew Gardens', 'Maspeth', 'Middle Village', 'Rego Park',
    'Ridgewood', 'Wyckoff Heights',
    'Bayside', 'Bay Terrace', 'Bayside Hills', 'Fort Totten', 'Oakland Gardens', 'Bellerose',
    'College Point', 'Douglaston', 'Douglas Bay', 'Douglas Manor', 'Douglaston Hill',
    'Douglaston Park', 'Winchester Estates', 'Little Neck', 'Pines', 'Little Neck Hills',
    'Westmoreland', 'Flushing', 'Broadway-Flushing', 'Bowne Park', 'Chinatown', 'Downtown Flushing',
    'Koreatown', 'Linden Hill', 'Murray Hill', 'Willets Point', 'Pomonok', 'Electchester',
    'Queensboro Hill', 'Floral Park', 'Auburndale', 'Kew Gardens Hills', 'Parkway Village',
    'Fresh Meadows', 'Hillcrest', 'Utopia', 'Glen Oaks', 'North Shore Towers', 'Whitestone',
    'Beechhurst', 'Clearview', 'Malba', 'Bellaire', 'Brookville', 'Cambria Heights', 'Hollis',
    'Hollis Hills', 'Holliswood', 'Jamaica', 'Jamaica Estates', 'Jamaica Hills', 'South Jamaica',
    'Rochdale Village', 'St. Albans', 'Laurelton', 'Queens Village', 'Rosedale', 'Meadowmere',
    'Warnerville', 'Springfield Gardens', 'Arverne', 'Bayswater', 'Belle Harbor', 'Breezy Point',
    'Broad Channel', 'Edgemere', 'Far Rockaway', 'Hammels', 'Neponsit', 'Rockaway Beach',
    'Rockaway Park', 'Roxbury', 'Seaside'
]


In [ ]:
Brooklyn = ['East Williamsburg',
            'South Williamsburg',
            'Williamsburg - North Side'
            'Greenpoint',
            'Downtown Brooklyn',
            'Fort Greene',
            'Bedford Stuyvesant',
            'Bushwick',
            'East New York',
            'Cypress Hills',
            'Park Slope',
            'Carroll Gardens',
            'Sunset Park',
            'Windsor Terrace',
            'Crown Heights',
            'Bay Ridge',
            'Dyker Heights',
            'Bensonhurst',
            'Bath Beach',
            'Borough Park',
            'Kensington',
            'Coney Island',
            'Brighton Beach',
            'Flatbush',
            'Midwood',
            'Sheepshead Bay',
            'Gravesend',
            'Ocean Hill',
            'Brownsville',
            'East Flatbush',
            'Canarsie',
            'Flatlands',
            'Prospect Park',
            'Jamaica Bay']

In [ ]:
staten_island = [
    'Annadale', 'Arden Heights', 'Arlington', 'Arrochar', 'Bay Terrace',
    'Bloomfield', 'Brighton Heights', 'Bulls Head', 'Castleton Corners',
    'Charleston', 'Chelsea', 'Clifton', 'Concord', 'Dongan Hills',
    'Egbertville', 'Elm Park', 'Eltingville', 'Emerson Hill',
    'Fort Wadsworth', 'Graniteville', 'Grant City', 'Grasmere',
    'Great Kills', 'Greenridge', 'Grymes Hill', 'Hamilton Park',
    'Heartland Village', 'Huguenot', 'Lighthouse Hill', 'Livingston',
    'Manor Heights', 'Mariners Harbor', 'Meiers Corners',
    ]


In [ ]:
staten_island = ['Midland Beach',
    'New Brighton', 'New Dorp', 'New Springville', 'Oakwood', 'Old Place',
    'Old Town', 'Pleasant Plains', 'Port Ivory', 'Port Richmond',
    'Prince\'s Bay', 'Randall Manor', 'Richmond Valley', 'Richmondtown',
    'Rosebank', 'Rossville', 'Saint George', 'Sandy Ground',
    'Shore Acres', 'Silver Lake', 'South Beach', 'Stapleton',
    'Stapleton Heights', 'Sunnyside', 'Todt Hill', 'Tompkinsville',
    'Tottenville', 'Tottenville Beach', 'Travis', 'Ward Hill',
    'West New Brighton', 'Westerleigh', 'Willowbrook', 'Woodrow']

In [ ]:
nyc = [[] for i in range(len(Queens))]

In [ ]:
# Function to draw in data for each neighborhood:
for x in range(len(Queens)):
    print('---------------------------------------------')
    print('Gathering Data for {}'.format(Queens[x]))
    print('---------------------------------------------')


    for y in range(12):
        location = Queens[x]+', New York City'
        term = "Restaurants"
        search_limit = 20
        offset = 20 * y
        categories = "(restaurants, All)"
        sort_by = 'distance'

        url_params = {
                        'location': location.replace(' ', '+'),
                        'term' : term,
                        'limit': search_limit,
                        'offset': offset,
                        'categories': categories,
                        'sorty_by': sort_by
                    }

        response = requests.get(url, headers=headers, params=url_params)
        print('***** {} Restaurants #{} - #{} ....{}'.format(Queens[x],
                                                             offset+1, offset+search_limit,
                                                             response))
        nyc[x].append(response)

print(response)
print(type(response.text))
print(response.json().keys())
print(response.text[:1000])

---------------------------------------------
Gathering Data for Beechhurst
---------------------------------------------
***** Beechhurst Restaurants #1 - #20 ....<Response [200]>
***** Beechhurst Restaurants #21 - #40 ....<Response [200]>
***** Beechhurst Restaurants #41 - #60 ....<Response [200]>
***** Beechhurst Restaurants #61 - #80 ....<Response [200]>
***** Beechhurst Restaurants #81 - #100 ....<Response [200]>
***** Beechhurst Restaurants #101 - #120 ....<Response [200]>
***** Beechhurst Restaurants #121 - #140 ....<Response [200]>
***** Beechhurst Restaurants #141 - #160 ....<Response [200]>
***** Beechhurst Restaurants #161 - #180 ....<Response [200]>
***** Beechhurst Restaurants #181 - #200 ....<Response [200]>
***** Beechhurst Restaurants #201 - #220 ....<Response [200]>
***** Beechhurst Restaurants #221 - #240 ....<Response [200]>
---------------------------------------------
Gathering Data for Clearview
---------------------------------------------
***** Clearview Restaur

In [ ]:
for x in range(len(Queens)):
  df = pd.DataFrame()
  for y in range(12):
      # Extract JSON data from the response
      data = nyc[x][y].json()
      df_temp = pd.DataFrame.from_dict(data['businesses']) # Now access 'businesses' from the JSON
      # Check if df_temp is not empty before adding the 'neighborhood' column
      if not df_temp.empty:
          df_temp.loc[:,'neighborhood'] = Queens[x]

      neighborhood = Queens[x].replace(' ', '_')
      df = pd.concat([df, df_temp], ignore_index=True) # Use pd.concat to append dataframes
  df.to_csv(f'Queens_{neighborhood}.csv')

# LA

In [ ]:
dataframe.to_csv('la_34_3906.csv')

In [ ]:
los_angeles = ['Beverly Hills',
'Bel Air',
'Brentwood',
'Century City',
'Chinatown',
'Downtown',
'Echo Park',
'Hollywood',
'Koreatown',
'Little Tokyo',
'Los Feliz',
'Malibu',
'Santa Monica',
'Silver Lake',
'Venice',
'Westwood']

In [ ]:
len(los_angeles)

16

## Neighborhoods:138

In [ ]:
los_angeles = [
    "Adams Normandie", "Alsace", "Angeles National Forest", "Angelino Heights", "Arleta",
    "Atwater Village", "Baldwin Hills", "Bel Air", "Beverly Crest", "Beverlywood",
    "Boyle Heights", "Brentwood", "Brookside", "Cadillac-Corning", "Canoga Park",
    "Carthay", "Central", "Century City", "Century Palms-Grove", "Chatsworth",
    "Cheviot Hills", "Chinatown", "Cloverdale-Cochran", "Country Club Park", "Crenshaw District",
    "Crestview", "Del Rey", "Downtown", "Eagle Rock", "East Hollywood",
    "Echo Park", "El Sereno", "Elysian Park", "Elysian Valley", "Encino",
    "Exposition",

    "Exposition Park", "Faircrest Heights", #"Figueroa Park Square",
               "Florence-Firestone", "Glassell Park", "Gramercy Park", #gramercy place
               "Granada Hills", "Green Meadows", "Hancock Park","Harbor City", "Harbor Gateway",
               "Harbor Pines", "Harvard Heights", "Harvard Park", "Highland Park",
               "Historic Filipinotown", "Hollywood", "Hollywood Hills", "Hyde Park",
               "Jefferson Park", "Koreatown", #"Lafayette Square",
               "Lake Balboa", "Lake View Terrace","Leimert Park", "Lincoln Heights",
               "Little Armenia", "Little Bangladesh", "Little Tokyo",
               "Longwood", "Los Feliz", "Manchester Square", #"Mandeville Canyon", "Marina Peninsula", "Melrose",
               "Mar Vista", "Mid-City", "Miracle Mile", "Mission Hills", "Mount Washington", #Mt. Washington
               "North Hills", "North Hollywood", "Northridge", "Pacific Palisades","Pacoima", #"Palisades Highlands",
               "Palms",  #"Park La Brea", "Reseda Ranch"

    "Panorama City","Pico-Union", "Playa del Rey", "Playa Vista", "Porter Ranch", "Rancho Park", #"Regent Square", "Reynier Village",
"Reseda", "San Pedro",
    "Shadow Hills", "Sherman Oaks", "Silver Lake",  "South Park", #"South Carthay", "St Elmo Village",
    "Studio City", "Sun Valley", "Sunland", 'Sycamore Terrace', #"Sycamore Square", "Temple-Beaudry", "Thai Town",
    "Sylmar", "Tarzana", "Toluca Lake",
    "Toluca Terrace", "Toluca Woods", "Tujunga", "University Park", #"University Hills",
    "Valley Glen", "Valley Village", "Van Nuys", "Venice", "Vermont Knolls",
    "Vermont Square", "Vermont Vista",  "Watts", # "Vernon Central", "Victoria Park","Wellington Square",
    "West Adams", "West Hills", "West Los Angeles", "West Vernon",
    "Westchester", "Westlake", "Westwood", 'Arts District', #"Wholesale District",
    "Wilmington",
    "Wilshire Center", "Winnetka", "Woodland Hills"
]


In [ ]:
la = [[] for i in range(len(los_angeles))]

In [ ]:
# Function to draw in data for each neighborhood:
for x in range(len(los_angeles)):
    print('---------------------------------------------')
    print('Gathering Data for {}'.format(los_angeles[x]))
    print('---------------------------------------------')


    for y in range(12):
        location = los_angeles[x]+', Los Angeles, CA'
        term = "Restaurants"
        search_limit = 20
        offset = 20 * y
        categories = "(restaurants, All)"
        sort_by = 'distance'

        url_params = {
                        'location': location.replace(' ', '+'),
                        'term' : term,
                        'limit': search_limit,
                        'offset': offset,
                        'categories': categories,
                        'sorty_by': sort_by
                    }

        response = requests.get(url, headers=headers, params=url_params)
        print('***** {} Restaurants #{} - #{} ....{}'.format(los_angeles[x],
                                                             offset+1, offset+search_limit,
                                                             response))
        la[x].append(response)

print(response)
print(type(response.text))
print(response.json().keys())
print(response.text[:1000])

---------------------------------------------
Gathering Data for Century Palms-Grove
---------------------------------------------
***** Century Palms-Grove Restaurants #1 - #20 ....<Response [200]>
***** Century Palms-Grove Restaurants #21 - #40 ....<Response [200]>
***** Century Palms-Grove Restaurants #41 - #60 ....<Response [200]>
***** Century Palms-Grove Restaurants #61 - #80 ....<Response [200]>
***** Century Palms-Grove Restaurants #81 - #100 ....<Response [200]>
***** Century Palms-Grove Restaurants #101 - #120 ....<Response [200]>
***** Century Palms-Grove Restaurants #121 - #140 ....<Response [200]>
***** Century Palms-Grove Restaurants #141 - #160 ....<Response [200]>
***** Century Palms-Grove Restaurants #161 - #180 ....<Response [200]>
***** Century Palms-Grove Restaurants #181 - #200 ....<Response [200]>
***** Century Palms-Grove Restaurants #201 - #220 ....<Response [200]>
***** Century Palms-Grove Restaurants #221 - #240 ....<Response [200]>
---------------------------

In [ ]:
for x in range(len(los_angeles)):
  df = pd.DataFrame()
  for y in range(12):
      # Extract JSON data from the response
      data = la[x][y].json()
      df_temp = pd.DataFrame.from_dict(data['businesses']) # Now access 'businesses' from the JSON

      # Check if df_temp is not empty before adding the 'neighborhood' column
      if not df_temp.empty:
          df_temp.loc[:,'neighborhood'] = los_angeles[x]

      neighborhood = los_angeles[x].replace(' ', '_')
      df = pd.concat([df, df_temp], ignore_index=True) # Use pd.concat to append dataframes
  df.to_csv(f'la_{neighborhood}.csv')

## Community Area

In [ ]:
los_angeles = [
    "Harbor Gateway",
    "Palms",
    "Mar Vista",
    "Del Rey",
    "Bel Air",
    "Beverly Crest",
    "Granada Hills",
    "Knollwood",
    "Mission Hills",
    "Panorama City",
    "North Hills",
    "Encino",
    "Tarzana",
    "Brentwood",
    "Pacific Palisades",
    "West Adams",
    "Baldwin Hills",
    "Leimert Park",
    "North Hollywood",
    "Valley Village",
    "Arleta",
    "Pacoima",
    "Northeast Los Angeles",
    "Venice",
    "Silver Lake",
    "Echo Park",
    "Elysian Valley",
    "San Pedro",
    "Reseda",
    "West Van Nuys",
    "Sun Valley",
    "La Tuna Canyon",
    "Sunland",
    "Tujunga",
    "Lake View Terrace",
    "Shadow Hills",
    # "East La Tuna Canyon",
    "Westwood",
    "West Los Angeles",
    "Hollywood",
    "Canoga Park",
    "Winnetka",
    "Woodland Hills",
    "West Hills",
    # "Central City North",
    "Chatsworth",
    "Porter Ranch",
    "Wilmington",
    "Harbor City",
    "Sylmar",
    "Mid-Wilshire",
    "Wilshire Park",
    "Century City",
    "Westlake",
    # "Port of Los Angeles",
    "Northridge",
    "Van Nuys",
    "Sherman Oaks",
    "Boyle Heights",
    "Studio City",
    "Toluca Lake",
    # "Cahuenga Pass",
    "Westchester",
    "Playa del Rey",
    "Los Angeles International Airport",
    "South Los Angeles",
    # "Southeast Los Angeles"
]


In [ ]:
len(los_angeles)

42

# Chicago

In [ ]:
chicago = [[] for i in range(len(chicago_city))]

In [ ]:
# Function to draw in data for each neighborhood:
for x in range(len(chicago_city)):
    print('---------------------------------------------')
    print('Gathering Data for {}'.format(chicago_city[x]))
    print('---------------------------------------------')


    for y in range(12):
        location = chicago_city[x]+', Chicago'
        term = "Restaurants"
        search_limit = 20
        offset = 20 * y
        categories = "(restaurants, All)"
        sort_by = 'distance'

        url_params = {
                        'location': location.replace(' ', '+'),
                        'term' : term,
                        'limit': search_limit,
                        'offset': offset,
                        'categories': categories,
                        'sorty_by': sort_by
                    }

        response = requests.get(url, headers=headers, params=url_params)
        print('***** {} Restaurants #{} - #{} ....{}'.format(chicago_city[x],
                                                             offset+1, offset+search_limit,
                                                             response))
        chicago[x].append(response)

print(response)
print(type(response.text))
print(response.json().keys())
print(response.text[:1000])

---------------------------------------------
Gathering Data for South Chicago
---------------------------------------------
***** South Chicago Restaurants #1 - #20 ....<Response [200]>
***** South Chicago Restaurants #21 - #40 ....<Response [200]>
***** South Chicago Restaurants #41 - #60 ....<Response [200]>
***** South Chicago Restaurants #61 - #80 ....<Response [200]>
***** South Chicago Restaurants #81 - #100 ....<Response [200]>
***** South Chicago Restaurants #101 - #120 ....<Response [200]>
***** South Chicago Restaurants #121 - #140 ....<Response [200]>
***** South Chicago Restaurants #141 - #160 ....<Response [200]>
***** South Chicago Restaurants #161 - #180 ....<Response [200]>
***** South Chicago Restaurants #181 - #200 ....<Response [200]>
***** South Chicago Restaurants #201 - #220 ....<Response [200]>
***** South Chicago Restaurants #221 - #240 ....<Response [200]>
---------------------------------------------
Gathering Data for South Deering
--------------------------

In [ ]:
df = pd.DataFrame()
for x in range(len(chicago_city)):
  for y in range(12):
    # Extract JSON data from the response
    data = chicago[x][y].json()
    df_temp = pd.DataFrame.from_dict(data['businesses']) # Now access 'businesses' from the JSON
    df_temp.loc[:,'neighborhood'] = chicago_city[x]
    neighborhood = chicago_city[x].replace(' ', '_')
    df = pd.concat([df, df_temp], ignore_index=True) # Use pd.concat to append dataframes
  df.to_csv(f'chicago_{neighborhood}.csv')

In [ ]:
chicago_city = [
    "Rogers Park", "West Ridge", "Uptown", "Lincoln Square", "North Center",
    "Lake View", "Lincoln Park", "Near North Side", "Edison Park", "Norwood Park",
    "Jefferson Park", "Forest Glen", "North Park", "Albany Park", "Portage Park",
    "Irving Park", "Dunning", "Montclare", "Belmont Cragin", "Hermosa",
    "Avondale", "Logan Square", "Humboldt Park", "West Town", "Austin",
    "West Garfield Park", "East Garfield Park", "Near West Side", "North Lawndale",
    "Little Village", "Lower West Side", "The Loop", "Near South Side", "Armour Square",
    "Douglas", "Oakland", "Grand Boulevard", "Kenwood", "Washington Park", #"Fuller Park",
    "Hyde Park", "Woodlawn", "South Shore",  "Avalon Park", "South Chicago",
    "Burnside", "Calumet Heights", "Roseland", "Pullman", "South Deering",
    "East Side", "West Pullman", "Riverdale", "Hegewisch", "Garfield Ridge",
    "Archer Heights", "Brighton Park", "McKinley Park", "Bridgeport", "New City",
    "West Elsdon", "Gage Park", "Clearing", "West Lawn", "Chicago Lawn",
    "West Englewood", "Englewood", "Greater Grand Crossing", "Ashburn", "Auburn Gresham",
    "Beverly", "Washington Heights", "Mount Greenwood", "Morgan Park", "O'Hare",
    "Edgewater"
]


# Houston

In [ ]:
houston_city = [
    'Acres Home', 'Addicks Park Ten', 'Afton Oaks', 'River Oaks', 'Alief',
    'Astrodome Area', 'Braeburn', 'Braeswood', 'Brays Oaks', 'Briar Forest',
    'Carverdale', 'Central Northwest', 'Central Southwest', 'Clear Lake',
    'Clinton Park', 'Tri-Community', 'Denver Harbor','Port Houston', 'Downtown',
    'East Houston', 'East Little York','Homestead', 'Eastex-Jensen Area',
    'Edgebrook Area', 'El Dorado','Oates Prairie', 'Eldridge','West Oaks',
    'Fairbanks','Northwest Crossing', 'Fondren Gardens', 'Fort Bend Houston',
    'Fourth Ward', 'Golfcrest','Bellfort','Reveille', 'Greater Eastwood',
    'Greater Fifth Ward', 'Greater Greenspoint', 'Greater Heights',
    'Greater Hobby Area', 'Greater Inwood', 'Greater OST','South Union',
    'Greater Third Ward', 'Greater Uptown', 'Greenway','Upper Kirby',
    'Gulfgate Riverview','Pine Valley',
    'Gulfton', #'Harrisburg', #'Manchester','Smith Addition',
    'Hidden Valley', 'Hunterwood', 'IAH Airport Area', 'Independence Heights',
    'Kashmere Gardens', 'Kingwood', 'Lake Houston', 'Langwood',
    'Lawndale','Wayside', 'Lazybrook',
    'Timbergrove', 'MacGregor',
    'Magnolia Park', 'Meadowbrook','Allendale', 'Medical Center',
    'Memorial', 'Meyerland', 'Mid West', 'Midtown', 'Minnetex',
    'Museum Park', 'Near Northside', 'Near Southwest', 'Neartown-Montrose',
    'Northshore', 'Northside','Northline', 'Park Place', 'Pecan Park',
    'Pleasantville', 'Second Ward', 'Settegast', 'Sharpstown',
    'South Acres','Crestmont Park', 'South Belt','Ellington', 'South Main',
    'South Park', 'Spring Branch Central', 'Spring Branch East',
    'Spring Branch North', 'Spring Branch West', 'Sunnyside',
    'Trinity/Houston Gardens'
]


In [ ]:
houston = [[] for i in range(len(houston_city))]

In [ ]:
# Function to draw in data for each neighborhood:
for x in range(len(houston_city)):
    print('---------------------------------------------')
    print('Gathering Data for {}'.format(houston_city[x]))
    print('---------------------------------------------')


    for y in range(12):
        location = houston_city[x]+', Houston'
        term = "Restaurants"
        search_limit = 20
        offset = 20 * y
        categories = "(restaurants, All)"
        sort_by = 'distance'

        url_params = {
                        'location': location.replace(' ', '+'),
                        'term' : term,
                        'limit': search_limit,
                        'offset': offset,
                        'categories': categories,
                        'sorty_by': sort_by
                    }

        response = requests.get(url, headers=headers, params=url_params)
        print('***** {} Restaurants #{} - #{} ....{}'.format(houston_city[x],
                                                             offset+1, offset+search_limit,
                                                             response))
        houston[x].append(response)

print(response)
print(type(response.text))
print(response.json().keys())
print(response.text[:1000])

---------------------------------------------
Gathering Data for Timbergrove
---------------------------------------------
***** Timbergrove Restaurants #1 - #20 ....<Response [200]>
***** Timbergrove Restaurants #21 - #40 ....<Response [200]>
***** Timbergrove Restaurants #41 - #60 ....<Response [200]>
***** Timbergrove Restaurants #61 - #80 ....<Response [200]>
***** Timbergrove Restaurants #81 - #100 ....<Response [200]>
***** Timbergrove Restaurants #101 - #120 ....<Response [200]>
***** Timbergrove Restaurants #121 - #140 ....<Response [200]>
***** Timbergrove Restaurants #141 - #160 ....<Response [200]>
***** Timbergrove Restaurants #161 - #180 ....<Response [200]>
***** Timbergrove Restaurants #181 - #200 ....<Response [200]>
***** Timbergrove Restaurants #201 - #220 ....<Response [200]>
***** Timbergrove Restaurants #221 - #240 ....<Response [200]>
---------------------------------------------
Gathering Data for MacGregor
---------------------------------------------
***** MacG

In [ ]:
for x in range(len(houston_city)):
  df = pd.DataFrame()
  for y in range(12):
      # Extract JSON data from the response
      data = houston[x][y].json()
      df_temp = pd.DataFrame.from_dict(data['businesses']) # Now access 'businesses' from the JSON

      # Check if df_temp is not empty before adding the 'neighborhood' column
      if not df_temp.empty:
          df_temp.loc[:,'neighborhood'] = houston_city[x]

      neighborhood = houston_city[x].replace(' ', '_')
      df = pd.concat([df, df_temp], ignore_index=True) # Use pd.concat to append dataframes
  df.to_csv(f'houston_{neighborhood}.csv')